# LSTM+Transformer 工业时序预测模型 - PAI-DSW 快速启动

本Notebook提供了在PAI-DSW环境中快速部署和运行LSTM+Transformer时序预测模型的完整流程。

## 📋 运行环境要求
- PAI-DSW实例 (推荐GPU实例)
- Python 3.8+
- CUDA支持 (推荐)

## 🚀 快速开始

## 1. 环境初始化

In [ ]:
# 检查当前环境
import os
import sys

print(f"当前工作目录: {os.getcwd()}")
print(f"Python版本: {sys.version}")

# 设置PAI-DSW环境
if os.path.exists('/mnt/workspace'):
    os.chdir('/mnt/workspace')
    sys.path.insert(0, '/mnt/workspace')
    print("✅ PAI-DSW环境设置完成")
else:
    print("⚠️  未检测到PAI-DSW环境，使用当前目录")

print(f"工作目录: {os.getcwd()}")

In [ ]:
# 运行完整的环境初始化
print("🚀 开始环境初始化...")
exec(open('setup_pai_dsw.py').read())

## 2. 检查GPU和深度学习环境

In [ ]:
# 检查PyTorch和CUDA
import torch
import numpy as np

print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU数量: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  显存: {torch.cuda.get_device_properties(i).total_memory / 1e9:.1f} GB")
        
    # 测试GPU
    device = torch.device('cuda')
    x = torch.randn(1000, 1000).to(device)
    y = torch.randn(1000, 1000).to(device)
    z = torch.mm(x, y)
    print("✅ GPU测试通过")
else:
    print("⚠️  将使用CPU进行训练")
    device = torch.device('cpu')

print(f"使用设备: {device}")

## 3. 数据准备和预处理

In [ ]:
# 检查数据文件
import glob

print("🔍 检查数据文件...")
data_paths = [
    'data/raw/*.csv',
    'data/*.csv',
    '/mnt/workspace/data/*.csv'
]

data_files = []
for path in data_paths:
    files = glob.glob(path)
    data_files.extend(files)

if data_files:
    print("✅ 发现数据文件:")
    for file in data_files:
        print(f"  - {file}")
        
    # 显示第一个数据文件的基本信息
    import pandas as pd
    
    df = pd.read_csv(data_files[0])
    print(f"\n数据集信息:")
    print(f"  形状: {df.shape}")
    print(f"  列名: {list(df.columns)}")
    print(f"  前5行:")
    print(df.head())
else:
    print("❌ 未发现数据文件!")
    print("请上传CSV格式的训练数据到以下目录之一:")
    print("  - data/raw/")
    print("  - data/")

In [ ]:
# 运行数据预处理 (如果数据文件存在)
if data_files:
    print("📊 开始数据预处理...")
    !python src/data/preprocess.py
    print("✅ 数据预处理完成!")
else:
    print("⚠️  跳过数据预处理 - 请先上传数据文件")

## 4. 模型训练

In [ ]:
# 检查配置文件
import yaml

config_path = 'configs/lstm_transformer.yaml'
if os.path.exists(config_path):
    with open(config_path, 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    print("✅ 配置文件加载成功")
    print(f"模型配置: {config['model']}")
    print(f"训练配置: {config['training']}")
else:
    print(f"❌ 配置文件不存在: {config_path}")

In [ ]:
# 开始模型训练 (这可能需要较长时间)
import subprocess
import time

if data_files and os.path.exists(config_path):
    print("🚀 开始模型训练...")
    print("注意: 训练可能需要几分钟到几小时，请耐心等待")
    
    start_time = time.time()
    
    # 运行训练
    result = subprocess.run([
        'python', 'src/training/train.py', 
        '--config', config_path
    ], capture_output=True, text=True)
    
    end_time = time.time()
    training_time = end_time - start_time
    
    print(f"训练耗时: {training_time/60:.1f} 分钟")
    
    if result.returncode == 0:
        print("✅ 模型训练完成!")
        print(result.stdout)
    else:
        print("❌ 模型训练失败:")
        print(result.stderr)
else:
    print("⚠️  跳过模型训练 - 请先完成数据预处理")

## 5. 训练进度监控

In [ ]:
# 实时查看训练历史
import matplotlib.pyplot as plt
import json

history_path = 'results/training_history.json'
if os.path.exists(history_path):
    with open(history_path, 'r') as f:
        history = json.load(f)
    
    # 绘制训练曲线
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Loss curves
    axes[0, 0].plot(history['train_loss'], label='Train Loss')
    axes[0, 0].plot(history['val_loss'], label='Val Loss')
    axes[0, 0].set_title('Loss Curves')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    # MAE curves
    axes[0, 1].plot(history['train_mae'], label='Train MAE')
    axes[0, 1].plot(history['val_mae'], label='Val MAE')
    axes[0, 1].set_title('MAE Curves')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # R² curves
    if 'train_r2' in history:
        axes[1, 0].plot(history['train_r2'], label='Train R²')
        axes[1, 0].plot(history['val_r2'], label='Val R²')
        axes[1, 0].set_title('R² Curves')
        axes[1, 0].legend()
        axes[1, 0].grid(True)
    
    # Learning rate
    if 'learning_rate' in history:
        axes[1, 1].plot(history['learning_rate'], label='Learning Rate')
        axes[1, 1].set_title('Learning Rate Schedule')
        axes[1, 1].legend()
        axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # 显示最佳指标
    print("🏆 训练结果摘要:")
    print(f"最终训练Loss: {history['train_loss'][-1]:.6f}")
    print(f"最终验证Loss: {history['val_loss'][-1]:.6f}")
    print(f"最佳验证MAE: {min(history['val_mae']):.6f}")
    if 'val_r2' in history:
        print(f"最佳验证R²: {max(history['val_r2']):.6f}")
else:
    print("⚠️  训练历史文件不存在，请先完成模型训练")

## 6. 模型预测和评估

In [ ]:
# 检查训练好的模型
model_path = 'models/best_model.pt'
if os.path.exists(model_path):
    print(f"✅ 发现训练好的模型: {model_path}")
    
    # 显示模型文件信息
    model_size = os.path.getsize(model_path) / (1024 * 1024)
    print(f"模型文件大小: {model_size:.2f} MB")
    
    # 加载模型检查点信息
    checkpoint = torch.load(model_path, map_location='cpu')
    if 'epoch' in checkpoint:
        print(f"训练轮数: {checkpoint['epoch']}")
    if 'val_loss' in checkpoint:
        print(f"验证损失: {checkpoint['val_loss']:.6f}")
    if 'val_mae' in checkpoint:
        print(f"验证MAE: {checkpoint['val_mae']:.6f}")
else:
    print("❌ 未发现训练好的模型，请先完成模型训练")

In [ ]:
# 运行模型预测
if os.path.exists(model_path):
    print("🔮 开始模型预测...")
    
    !python src/inference/predict.py \
        --config configs/lstm_transformer.yaml \
        --model_path models/best_model.pt \
        --datasets test \
        --generate_viz
    
    print("✅ 模型预测完成!")
else:
    print("⚠️  跳过模型预测 - 请先完成模型训练")

## 7. 结果可视化

In [ ]:
# 查看预测结果可视化
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

results_dir = Path('results')
if results_dir.exists():
    # 查找可视化图片
    viz_files = list(results_dir.glob('*.png'))
    
    if viz_files:
        print(f"📊 发现 {len(viz_files)} 个可视化文件:")
        
        # 显示预测结果图
        for viz_file in viz_files[:4]:  # 显示前4个图片
            print(f"\n📈 {viz_file.name}:")
            
            try:
                from IPython.display import Image, display
                display(Image(str(viz_file)))
            except:
                print(f"图片路径: {viz_file}")
    else:
        print("❌ 未发现可视化文件")
else:
    print("❌ 结果目录不存在")

In [ ]:
# 加载并显示预测指标
metrics_path = 'results/evaluation_metrics.json'
if os.path.exists(metrics_path):
    with open(metrics_path, 'r') as f:
        metrics = json.load(f)
    
    print("📊 预测评估指标:")
    for dataset_name, dataset_metrics in metrics.items():
        print(f"\n{dataset_name.upper()} 数据集:")
        for metric_name, value in dataset_metrics.items():
            if isinstance(value, float):
                if metric_name in ['mape', 'smape']:
                    print(f"  {metric_name.upper()}: {value:.2f}%")
                elif metric_name == 'r2':
                    print(f"  {metric_name.upper()}: {value:.4f}")
                else:
                    print(f"  {metric_name.upper()}: {value:.6f}")
else:
    print("⚠️  评估指标文件不存在")

## 8. ONNX模型导出

In [ ]:
# 导出ONNX模型
if os.path.exists(model_path):
    print("📦 开始ONNX模型导出...")
    
    !python src/inference/export_onnx.py \
        --model_path models/best_model.pt \
        --output_path models/model.onnx \
        --batch_size 1 \
        --sequence_length 96 \
        --input_size 1
    
    # 检查ONNX文件
    onnx_path = 'models/model.onnx'
    if os.path.exists(onnx_path):
        onnx_size = os.path.getsize(onnx_path) / (1024 * 1024)
        print(f"✅ ONNX模型导出成功!")
        print(f"ONNX文件大小: {onnx_size:.2f} MB")
    else:
        print("❌ ONNX模型导出失败")
else:
    print("⚠️  跳过ONNX导出 - 请先完成模型训练")

## 9. 项目总结

In [ ]:
# 项目完成度检查
print("🎯 项目完成度检查:")
print("=" * 50)

checks = [
    ("环境初始化", os.path.exists('pai_dsw_config.json')),
    ("数据预处理", len(glob.glob('data/processed/*.npz')) > 0),
    ("模型训练", os.path.exists('models/best_model.pt')),
    ("训练历史", os.path.exists('results/training_history.json')),
    ("模型预测", os.path.exists('results/evaluation_metrics.json')),
    ("可视化图表", len(list(Path('results').glob('*.png'))) > 0 if Path('results').exists() else False),
    ("ONNX导出", os.path.exists('models/model.onnx'))
]

completed = 0
for task, status in checks:
    status_icon = "✅" if status else "❌"
    print(f"{status_icon} {task}")
    if status:
        completed += 1

completion_rate = completed / len(checks) * 100
print(f"\n📈 项目完成度: {completion_rate:.1f}% ({completed}/{len(checks)})")

if completion_rate == 100:
    print("\n🎉 恭喜! 项目已完整部署并运行成功!")
elif completion_rate >= 70:
    print("\n👍 项目主要功能已完成，可以继续优化")
else:
    print("\n⚠️  项目仍有部分功能需要完善")

# 显示重要文件路径
print("\n📁 重要文件位置:")
important_files = [
    'models/best_model.pt',
    'models/model.onnx', 
    'results/evaluation_metrics.json',
    'results/training_history.json',
    'pai_dsw_config.json'
]

for file_path in important_files:
    if os.path.exists(file_path):
        file_size = os.path.getsize(file_path) / 1024
        if file_size > 1024:
            print(f"  ✅ {file_path} ({file_size/1024:.1f} MB)")
        else:
            print(f"  ✅ {file_path} ({file_size:.1f} KB)")
    else:
        print(f"  ❌ {file_path} (不存在)")

## 📚 后续操作建议

### 模型优化
1. **超参数调优**: 修改 `configs/lstm_transformer.yaml` 中的参数
2. **数据增强**: 在 `src/data/preprocess.py` 中添加数据增强策略
3. **模型架构**: 在 `src/models/` 中尝试不同的模型结构

### 生产部署
1. **ONNX推理**: 使用导出的ONNX模型进行高效推理
2. **批量预测**: 运行 `src/inference/batch_predict.py` 进行批量处理
3. **结果分析**: 使用 `src/inference/analyze_results.py` 进行深入分析

### 监控和维护
1. **性能监控**: 定期检查模型性能指标
2. **数据漂移**: 监控输入数据的分布变化
3. **模型更新**: 根据新数据定期重训练模型

---

**🎉 恭喜! 您已成功在PAI-DSW环境中部署了LSTM+Transformer工业时序预测模型!**